## Get Candlestick Data 

In [1]:
import requests
import calendar
import dateutil.parser as parser
from dateutil.relativedelta import relativedelta
from datetime import datetime, timezone
import yaml
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
now = datetime.now()

In [3]:
def convert_date(utc_time): 
    parsed_date = parser.parse(utc_time)
    var_date=parsed_date.date()
    var_time=parsed_date.time()
    var_f_time=var_time.hour
    var_julian_date=parsed_date.timetuple().tm_yday
    var_weekday=parsed_date.weekday()
    var_weekday_name=calendar.day_name[parsed_date.weekday()]
    return var_date, var_time, var_f_time, var_julian_date, var_weekday, var_weekday_name

### The Configs for Run:

In [4]:
with open ('config.yml') as ymlfile:
    cfg = yaml.safe_load(ymlfile)
    oanda_api_key = cfg['creds']['oanda_api']
    account_number = cfg['creds']['account_number'] 

### Select the Currency Pair

In [5]:
Load_10K_Records=True

#  'EUR_USD','USD_CAD','EUR_GBP','EUR_AUD','EUR_CHF',
#  'GBP_USD','GBP_CHF','GBP_NZD','GBP_AUD','GBP_CAD',
#  'AUD_CAD','AUD_CHF','AUD_NZD','NZD_USD','EUR_CAD',
#  'USD_CHF','CAD_CHF','NZD_CAD','AUD_USD','EUR_NZD',
#  'NZD_CHF',


currency_pairs = ["USD_CAD"]


timeframe = "H4"
#D #H1 #H4 M30
# https://developer.oanda.com/rest-live-v20/instrument-df/#CandlestickGranularity
price_char = "M"
#M(midpoint candles) #B(bid candles) #A(ask candles) #BA
price_com = "mid"
#mid #bid #ask

# def of OANDA request variable
provider_api_url = 'https://api-fxpractice.oanda.com/v3/accounts/{}/orders'.format(account_number)
request_headers = {
    "Authorization": oanda_api_key,
    "Accept-Datetime-Format": "RFC3339",
    "Connection": "Keep-Alive",
    "Content-Type": "application/json;charset=UTF-8"
}


In [6]:
provider_authorization = 'Bearer {0}'.format(oanda_api_key)

headers = {
    'Content-Type': 'application/json',
    'Authorization': provider_authorization,
}

# Get Candlesticks Data

In [7]:
params_count = (
    ('price', price_char),
    ('count', '5000'),
    ('granularity', timeframe),
)


In [8]:
for pair in currency_pairs:
    first_response = requests.get('https://api-fxpractice.oanda.com/v3/instruments/{}/candles'.format(pair), 
                            headers=headers,
                            params=params_count).json()

In [9]:
if Load_10K_Records:
    datetime_object = parser.parse(first_response['candles'][0]['time'])
    date= datetime_object - relativedelta(years=3)  
    from_date = date.replace(tzinfo=timezone.utc).timestamp()
    params_date = (
        ('count', '5000'),
        ('price', price_char),
        ('from', from_date),
        ('granularity', timeframe),)

    second_response = requests.get('https://api-fxpractice.oanda.com/v3/instruments/{}/candles'.format(pair),
                                   headers=headers,
                                   params=params_date).json()
            
    first_response= first_response['candles']  
    second_response= second_response['candles']
    second_response.extend(first_response)
    response=second_response
else:
    response=first_response['candles']

In [10]:
filename = "{}_{}.csv".format(pair, timeframe)
output = []
all_candlesticks = response

for i in range (len(all_candlesticks)):
    result= (convert_date(response[i]['time']))
    output.append([(result[0]),(result[1]),(result[2]),(result[3]),(result[4]),(result[5]),
                    response[i]['time'],
                    response[i]['volume'], 
                    response[i][price_com]['o'],
                    response[i][price_com]['h'],
                    response[i][price_com]['l'],
                    response[i][price_com]['c']])
    
output = pd.DataFrame(output)
output.columns = ['Date','Time','f_time','julian_date','Weekday','Weekday_Name','UTC_Time', 'Volume', 'Open', 'High', 'Low', 'Close']
data = output.to_csv(filename, header = True, index = False)
data = pd.read_csv(filename)

In [11]:
data = data.drop_duplicates()
data = data.to_csv(filename, header = True, index = False)
data = pd.read_csv(filename)

In [12]:
data.shape

(9658, 12)

In [13]:
data.head(5)

,Date,Time,f_time,julian_date,Weekday,Weekday_Name,UTC_Time,Volume,Open,High,Low,Close
0,2015-10-30,05:00:00,5,303,4,Friday,2015-10-30T05:00:00.000000000Z,5792,1.31655,1.31686,1.31358,1.31452
1,2015-10-30,09:00:00,9,303,4,Friday,2015-10-30T09:00:00.000000000Z,9602,1.31453,1.31922,1.31258,1.31868
2,2015-10-30,13:00:00,13,303,4,Friday,2015-10-30T13:00:00.000000000Z,15320,1.31868,1.31930,1.30558,1.30662
3,2015-10-30,17:00:00,17,303,4,Friday,2015-10-30T17:00:00.000000000Z,8546,1.30662,1.30840,1.30598,1.30754
4,2015-11-01,22:00:00,22,305,6,Sunday,2015-11-01T22:00:00.000000000Z,4648,1.30786,1.30817,1.30634,1.30701


In [14]:
data.tail(5)

,Date,Time,f_time,julian_date,Weekday,Weekday_Name,UTC_Time,Volume,Open,High,Low,Close
9653,2022-01-10,10:00:00,10,10,0,Monday,2022-01-10T10:00:00.000000000Z,4348,1.26242,1.26677,1.26104,1.26640
9654,2022-01-10,14:00:00,14,10,0,Monday,2022-01-10T14:00:00.000000000Z,9149,1.26638,1.26966,1.26632,1.26942
9655,2022-01-10,18:00:00,18,10,0,Monday,2022-01-10T18:00:00.000000000Z,3958,1.26944,1.26978,1.26702,1.26762
9656,2022-01-10,22:00:00,22,10,0,Monday,2022-01-10T22:00:00.000000000Z,1914,1.26724,1.26818,1.26592,1.26664
9657,2022-01-11,02:00:00,2,11,1,Tuesday,2022-01-11T02:00:00.000000000Z,1042,1.26668,1.26692,1.26454,1.26508


## Simple Moving Average (SMA)

In [15]:
data['SMA_5'] = data['Close'].rolling(window=5).mean().round(4)
data['SMA_10'] = data['Close'].rolling(window=10).mean().round(4)
data['SMA_20'] = data['Close'].rolling(window=20).mean().round(4)

## Simple Moving Average Range

In [16]:
data['F_SMA_5'] = data['Close'] - data['SMA_5']
data['F_SMA_10'] = data['Close'] - data['SMA_10']
data['F_SMA_20'] = data['Close'] - data['SMA_20']

In [17]:
data = data.drop_duplicates()
data = data.to_csv(filename, header = True, index = False)
data = pd.read_csv(filename)

In [18]:
data.tail()

,Date,Time,f_time,julian_date,Weekday,Weekday_Name,UTC_Time,Volume,Open,High,Low,Close,SMA_5,SMA_10,SMA_20,F_SMA_5,F_SMA_10,F_SMA_20
9653,2022-01-10,10:00:00,10,10,0,Monday,2022-01-10T10:00:00.000000000Z,4348,1.26242,1.26677,1.26104,1.26640,1.2642,1.2672,1.2707,0.00220,-0.00080,-0.00430
9654,2022-01-10,14:00:00,14,10,0,Monday,2022-01-10T14:00:00.000000000Z,9149,1.26638,1.26966,1.26632,1.26942,1.2653,1.2670,1.2706,0.00412,0.00242,-0.00118
9655,2022-01-10,18:00:00,18,10,0,Monday,2022-01-10T18:00:00.000000000Z,3958,1.26944,1.26978,1.26702,1.26762,1.2660,1.2667,1.2705,0.00162,0.00092,-0.00288
9656,2022-01-10,22:00:00,22,10,0,Monday,2022-01-10T22:00:00.000000000Z,1914,1.26724,1.26818,1.26592,1.26664,1.2665,1.2662,1.2703,0.00014,0.00044,-0.00366
9657,2022-01-11,02:00:00,2,11,1,Tuesday,2022-01-11T02:00:00.000000000Z,1042,1.26668,1.26692,1.26454,1.26508,1.2670,1.2655,1.2697,-0.00192,-0.00042,-0.00462


## Price Range

In [19]:
data['O-H'] = data['Open'] - data['High']
data['O-L'] = data['Open'] - data['Low']
data['O-C'] = data['Open'] - data['Close']
data['H-L'] = data['High'] - data['Low']
data['H-C'] = data['High'] - data['Close']
data['L-C'] = data['Low'] - data['Close']

data['Direction'] = data['O-C'].apply(lambda x: 1 if x<0 else 0)

data['col_1'] = data['Open'] - data['Close']

for value in data['col_1']:   
    if value > 0:
        data['col_2'] = data['High'] - data['Open']
        data['col_3'] = data['Close'] - data['Low']
    else:
        data['col_2'] = data['High'] - data['Close']
        data['col_3'] = data['Open'] - data['Low']

#Two Previous Candlesticks 
data['col_4'] = data['col_1'].shift(1)
data['col_5'] = data['col_1'].shift(2)

In [20]:
data = data.dropna()
data = data.to_csv(filename, header = True, index = False)
data = pd.read_csv(filename)

In [21]:
data.tail()

,Date,Time,f_time,julian_date,Weekday,Weekday_Name,UTC_Time,Volume,Open,High,...,O-C,H-L,H-C,L-C,Direction,col_1,col_2,col_3,col_4,col_5
9634,2022-01-10,10:00:00,10,10,0,Monday,2022-01-10T10:00:00.000000000Z,4348,1.26242,1.26677,...,-0.00398,0.00573,0.00037,-0.00536,1,-0.00398,0.00435,0.00536,0.00190,-0.00046
9635,2022-01-10,14:00:00,14,10,0,Monday,2022-01-10T14:00:00.000000000Z,9149,1.26638,1.26966,...,-0.00304,0.00334,0.00024,-0.00310,1,-0.00304,0.00328,0.00310,-0.00398,0.00190
9636,2022-01-10,18:00:00,18,10,0,Monday,2022-01-10T18:00:00.000000000Z,3958,1.26944,1.26978,...,0.00182,0.00276,0.00216,-0.00060,0,0.00182,0.00034,0.00060,-0.00304,-0.00398
9637,2022-01-10,22:00:00,22,10,0,Monday,2022-01-10T22:00:00.000000000Z,1914,1.26724,1.26818,...,0.00060,0.00226,0.00154,-0.00072,0,0.00060,0.00094,0.00072,0.00182,-0.00304
9638,2022-01-11,02:00:00,2,11,1,Tuesday,2022-01-11T02:00:00.000000000Z,1042,1.26668,1.26692,...,0.00160,0.00238,0.00184,-0.00054,0,0.00160,0.00024,0.00054,0.00060,0.00182


In [22]:
data.shape

(9639, 30)